# Parte 1
## Análisis de Sentimientos

### Limpieza de Datos

#### Import and Data Shaping

In [241]:
import numpy as np 
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns

import os

# Preprocesado y modelado
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem import SnowballStemmer

import eli5

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [242]:
reviews = pd.read_csv('../csv/womens_clothing_reviews.csv', encoding='latin-1')
df = reviews.copy()
df

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [243]:
df.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')

In [244]:
df.drop(columns=['Unnamed: 0', 'Clothing ID'], inplace=True)
df

,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...
23481,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [245]:
reviews_and_target = df[['Review Text', 'Rating']]
reviews_and_target

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,4
1,Love this dress! it's sooo pretty. i happene...,5
2,I had such high hopes for this dress and reall...,3
3,"I love, love, love this jumpsuit. it's fun, fl...",5
4,This shirt is very flattering to all due to th...,5
...,...,...
23481,I was very happy to snag this dress at such a ...,5
23482,"It reminds me of maternity clothes. soft, stre...",3
23483,"This fit well, but the top was very see throug...",3
23484,I bought this dress for a wedding i have this ...,3


In [246]:
reviews_and_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review Text  22641 non-null  object
 1   Rating       23486 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 367.1+ KB


In [247]:
reviews_and_target['Good Ratings'] = reviews_and_target.Rating >= 4
reviews_and_target.drop(columns=['Rating'], inplace=True)
reviews_and_target

C:\Users\jonat\AppData\Local\Temp\ipykernel_60052\902374162.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_and_target['Good Ratings'] = reviews_and_target.Rating >= 4
C:\Users\jonat\AppData\Local\Temp\ipykernel_60052\902374162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_and_target.drop(columns=['Rating'], inplace=True)


,Review Text,Good Ratings
0,Absolutely wonderful - silky and sexy and comf...,True
1,Love this dress! it's sooo pretty. i happene...,True
2,I had such high hopes for this dress and reall...,False
3,"I love, love, love this jumpsuit. it's fun, fl...",True
4,This shirt is very flattering to all due to th...,True
...,...,...
23481,I was very happy to snag this dress at such a ...,True
23482,"It reminds me of maternity clothes. soft, stre...",False
23483,"This fit well, but the top was very see throug...",False
23484,I bought this dress for a wedding i have this ...,False


In [248]:
reviews_and_target['Good Ratings'] = reviews_and_target['Good Ratings'].astype(np.int8)
reviews_and_target

C:\Users\jonat\AppData\Local\Temp\ipykernel_60052\3936400925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_and_target['Good Ratings'] = reviews_and_target['Good Ratings'].astype(np.int8)


,Review Text,Good Ratings
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
...,...,...
23481,I was very happy to snag this dress at such a ...,1
23482,"It reminds me of maternity clothes. soft, stre...",0
23483,"This fit well, but the top was very see throug...",0
23484,I bought this dress for a wedding i have this ...,0


#### Verificación nulls

In [249]:
reviews_and_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Review Text   22641 non-null  object
 1   Good Ratings  23486 non-null  int8  
dtypes: int8(1), object(1)
memory usage: 206.5+ KB


Vemos que hay cantidades diferentes de reviews y ratings, por lo que vamos a sacar los datos inválidos.

In [250]:
reviews_and_target.dropna(inplace=True)
reviews_and_target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22641 entries, 0 to 23485
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Review Text   22641 non-null  object
 1   Good Ratings  22641 non-null  int8  
dtypes: int8(1), object(1)
memory usage: 375.9+ KB


C:\Users\jonat\AppData\Local\Temp\ipykernel_60052\1881422295.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_and_target.dropna(inplace=True)


Al hacer el `dropna()`, se están salteando índices. Para arreglar esto, vamos a resetear los índices con el método `reset_index()`.

In [251]:
reviews_and_target.reset_index(inplace=True)
reviews_and_target.drop(columns=['index'], inplace=True)
reviews_and_target

C:\Users\jonat\AppData\Local\Temp\ipykernel_60052\2140223640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_and_target.drop(columns=['index'], inplace=True)


,Review Text,Good Ratings
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
...,...,...
22636,I was very happy to snag this dress at such a ...,1
22637,"It reminds me of maternity clothes. soft, stre...",0
22638,"This fit well, but the top was very see throug...",0
22639,I bought this dress for a wedding i have this ...,0


### Entrenamiento con el método Regresión Logistica

In [252]:
tokenizer = ToktokTokenizer() 
nltk.download('stopwords')
stopWords = set(stopwords.words("english"))
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [253]:
def limpiar_texto(texto):
    texto = re.sub(r'\W', ' ', str(texto))
    texto = re.sub(r'\s+', ' ', texto, flags=re.I)
    return texto

def filtrar_stopword_digitos(tokens):
    return [token for token in tokens if token not in stopWords 
            and not token.isdigit()]

def stem_palabras(tokens):
    return [stemmer.stem(token) for token in tokens]

def tokenize(texto):
    texto_limpio = limpiar_texto(texto)
    tokens = [palabra for palabra in tokenizer.tokenize(texto_limpio) if len(palabra) > 1]
    tokens = filtrar_stopword_digitos(tokens)
    stems = stem_palabras(tokens)
    return stems

In [254]:
reviews = reviews_and_target['Review Text']
ratings = reviews_and_target['Good Ratings']

X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size = 0.3, random_state=2)

In [255]:
X_train.shape, y_train.shape

((15848,), (15848,))

In [256]:
X_test.shape, y_test.shape

((6793,), (6793,))

In [257]:
tfidf = TfidfVectorizer(
    tokenizer=tokenize,
    max_features=20000)  

tfidf.fit(X_train)

C:\Users\jonat\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:514: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_features=20000,
                tokenizer=<function tokenize at 0x00000234D2C8B370>)

In [258]:
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)

In [259]:
parameters = {'C':[50, 10, 5, 3, 1, 0.1, 0.25, 0.01]}

lr = LogisticRegression(random_state=2, solver='liblinear')

cv = KFold(n_splits=10, shuffle=True, random_state=2)

clf = GridSearchCV(lr, parameters, 
                   scoring='accuracy',
                   cv=cv,
                   refit=True,
                   verbose=2,
                   error_score='raise',
                   n_jobs=-1)

In [260]:
clf.fit(X_train, y_train)

Fitting 10 folds for each of 8 candidates, totalling 80 fits


GridSearchCV(cv=KFold(n_splits=10, random_state=2, shuffle=True),
             error_score='raise',
             estimator=LogisticRegression(random_state=2, solver='liblinear'),
             n_jobs=-1, param_grid={'C': [50, 10, 5, 3, 1, 0.1, 0.25, 0.01]},
             scoring='accuracy', verbose=2)

In [261]:
print('Mejor combinación de parámetros: %s ' % clf.best_params_)
print('CV Accuracy: %.3f' % clf.best_score_)

Mejor combinación de parámetros: {'C': 3} 
CV Accuracy: 0.873


In [262]:
best_clf = clf.best_estimator_
print('Test Accuracy: %.3f' % best_clf.score(X_test, y_test))

Test Accuracy: 0.876


In [263]:
eli5.show_weights(estimator=best_clf, 
                  feature_names= list(tfidf.get_feature_names()),
                 top=(10, 10))

Weight?,Feature
+8.149,perfect
+5.858,compliment
+5.536,great
+5.403,love
+5.301,comfort
+4.343,happi
+3.988,uniqu
+3.823,glad
+3.752,add
+3.734,sexi


In [264]:
model_pred = clf.predict(X_test)
f1_score(y_test, model_pred, average='micro')

0.8757544531134992